In [1]:
import os, pyodbc
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from sqlalchemy import create_engine

In [2]:
%pylab


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


c:\python27\arcgis10.2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
engine = create_engine("mssql+pyodbc://gt_KL_Master")

Query for Duplicate sample pairs

In [4]:
sql = ("Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'Ref',\
c.TypeDoublon  as 'Type',  coalesce(c.Col114, c.Col101) as 'Au_gpt', coalesce(f.Col114, f.Col101) as 'Au_gpt_orig', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
join tblAnalReg f on c.Reference = f.NuEchantillon \
where c.TypeQAQC = 2 and c.Col101 is not NULL and \
e.DateEmis > DATEADD(MONTH, -3, GETDATE()) and exists (Select * from tblAnalReg g join tblCertEchAnalyse h on \
g.idkey = h.idAnalReg where c.Reference = g.NuEchantillon) \
UNION \
Select  b.Nom as 'HOLEID', f.NuEchantillon as 'SAMPLEID', f.NuEchantillon as 'Ref', \
'Lab Duplicate'  as 'Type',  coalesce(f.Col115, f.Col102) as 'Au_gpt', \
coalesce(f.Col114, f.Col101) as 'Au_gpt_orig', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblAnalReg f on b.idkey = f.idNuSondage \
join tblCertEchAnalyse d on f.idkey = d.idAnalReg \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where (f.Col102 is not NULL or f.Col115 is not NULL) \
and f.Col102 not like '%NSS' and e.DateEmis > DATEADD(MONTH, -3, GETDATE())")

In [5]:
sql = ("Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'Ref',\
c.TypeDoublon  as 'Type',  coalesce(c.Col114, c.Col101) as 'Au_gpt', coalesce(f.Col114, f.Col101) as 'Au_gpt_orig', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
join tblAnalReg f on c.Reference = f.NuEchantillon \
where c.TypeQAQC = 2 and c.Col101 is not NULL and \
e.DateEmis between '2017-01-01' and '2017-07-01' and exists (Select * from tblAnalReg g join tblCertEchAnalyse h on \
g.idkey = h.idAnalReg where c.Reference = g.NuEchantillon) \
UNION \
Select  b.Nom as 'HOLEID', f.NuEchantillon as 'SAMPLEID', f.NuEchantillon as 'Ref', \
'Lab Duplicate'  as 'Type',  coalesce(f.Col115, f.Col102) as 'Au_gpt', \
coalesce(f.Col114, f.Col101) as 'Au_gpt_orig', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblAnalReg f on b.idkey = f.idNuSondage \
join tblCertEchAnalyse d on f.idkey = d.idAnalReg \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where (f.Col102 is not NULL or f.Col115 is not NULL) \
and f.Col102 not like '%NSS' and e.DateEmis between '2017-01-01' and '2017-07-01' ")

In [5]:
df = pd.read_sql(sql, engine)

In [6]:
df = df.reindex(columns = [u'HOLEID', u'SAMPLEID', u'Ref',u'Type', u'Au_gpt',u'Au_gpt_orig', u'Date',u'Au_val', u'Au_orig'])

In [7]:
df.Au_val = df.Au_gpt[(df.Au_gpt <> 'NIL') & (df.Au_gpt <> '<0.01') & (df.Au_gpt <> '<0.005') \
                      & (df.Au_gpt <> '< 0.005')].astype(np.float)
df.Au_orig = df.Au_gpt_orig[(df.Au_gpt_orig <> 'NIL') & (df.Au_gpt_orig <> '<0.01') \
                      & (df.Au_gpt_orig <> '<0.005') & (df.Au_gpt_orig <> '< 0.005')].astype(np.float)

In [8]:
df.loc[:,'Au_val'][df.Au_val.isnull()] = 0.005
df.loc[:,'Au_orig'][df.Au_orig.isnull()] = 0.005
df['Au_mean'] = (df.Au_val + df.Au_orig) /2.0
df['Au_diff'] = (df.Au_val - df.Au_orig)
df.Type[df['Type'] == 'FD'] = 'Field Duplicate'
df.Type[df['Type'] == 'CRD'] = 'Coarse Reject Duplicate'
df.Type[df['Type'] == 'PulpDuplicate'] = 'Umpire Lab'

c:\python27\arcgis10.2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\python27\arcgis10.2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\python27\arcgis10.2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\python27\arcgis10.2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is

X-Y plots

In [9]:
fig= plt.figure(facecolor = 'w', figsize = (12,12))
fig.suptitle('Duplicate Analyses for week of %s' % \
             (datetime.date.today() + datetime.timedelta(4 - \
              datetime.date.weekday(datetime.date.today()))).strftime('Friday, %B %d, %Y'), fontsize = 14)


index = 1

for d in ['Field Duplicate', 'Coarse Reject Duplicate', 'Lab Duplicate', 'Umpire Lab']:
    ax = fig.add_subplot(2,2,index)
    ax.plot((0,max(df.Au_val)*2), (0,max(df.Au_val)*2), 'r-')
    
    m, b = np.polyfit(df[df.Type == d].Au_val, df[df.Type == d].Au_orig, 1)
    n = np.linspace(0.005, max(df.Au_val) * 1.1, 500)
    ax.plot(n, (n * m) + b, 'b--')
    ax.scatter(df[df.Type == d].Au_val, df[df.Type == d].Au_orig, c = 'k')
    limit = max(max(df.Au_val), max(df.Au_orig)) * 1.1
    ax.set_xlim((0.005,limit))
    ax.set_ylim((0.005, limit))
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Duplicate (gpt)', fontsize = 10)
    ax.set_ylabel('Original (gpt)', fontsize = 10)
    xticks = ax.get_xticks()
    ax.set_xticklabels([x for x in xticks])
    yticks = ax.get_yticks()
    ax.set_yticklabels([y for y in yticks])
    
    # calculate and display correlation
    r2 = df[df.Type == d].Au_val.corr(df[df.Type == d].Au_orig) **2
    xpos = 10 ** (np.log10(0.005) - ((np.log10(0.005) - np.log10(limit))* 0.1) )
    ypos = 10 ** (np.log10(0.005) - ((np.log10(0.005) - np.log10(limit))* 0.85) )
    ax.annotate(u'r² = {:.3f}'.format(r2), (xpos, ypos) , bbox=dict(facecolor='w', alpha=0.2, ))
    xpos = 10 ** (np.log10(0.005) - ((np.log10(0.005) - np.log10(limit))* 0.5) )
    ypos = 10 ** (np.log10(0.005) - ((np.log10(0.005) - np.log10(limit))* 0.15) )
    ax.annotate(u'y = {:.5f} * x {}\n {:.5f}'.format(m, '-' if b < 0 else '+',abs(b)), (xpos, ypos), \
                bbox = dict(facecolor = 'b', alpha = 0.2, ), color = 'b')
    
    ax.set_title('Duplicate type %s, n = %d' % (d, df[df.Type == d]['HOLEID'].count()), fontsize = 12)
    index += 1
   

Absolute Relative Difference vs Mean plots

In [10]:
fig2, axes = plt.subplots(4,1, facecolor = 'w', figsize = (12, 14), sharex = True, sharey = True)
fig2.suptitle('Duplicate Analyses for week of %s' % \
             (datetime.date.today() + datetime.timedelta(4 - \
              datetime.date.weekday(datetime.date.today()))).strftime('Friday, %B %d, %Y'), fontsize = 14)
#fig2.figimage(logo, 690,1030, zorder=1)

index = 0
for d in ['Field Duplicate', 'Coarse Reject Duplicate', 'Lab Duplicate', 'Umpire Lab']:
    ax = axes[index]
    
    #Draw 3-sigma or 1% limit line
    xlimit = max(df.Au_mean) * 1.1
    lin = np.linspace(0.005, xlimit, 200)
    mlin = np.where((lin*0.01)> 0.01, lin*0.01, 0.01)
    ax.plot(lin,  mlin/lin * 100 , 'r--')
    
    #Plot average vs relative difference points
    ax.scatter(df[df.Type == d].Au_mean,abs(df[df.Type == d].Au_diff)/df[df.Type == d].Au_mean *100, c = 'w', marker = 'o', s = 32 )
    ax.scatter(df[df.Type == d].Au_mean,abs(df[df.Type == d].Au_diff)/df[df.Type == d].Au_mean *100, c = 'k', marker = '.', s = 8)
    
    #Set graph limits and log scale on x-axis
    xlim((0.005,xlimit))
    ylim((0, 200))
    ax.set_xscale('log')
    
    #Plot x-axis labels on last graph
    if d == 'Umpire Lab':
        ax.set_xlabel('Pair Mean (Au g/t)')
        xticks = ax.get_xticks()
        ax.set_xticklabels([x for x in xticks])
        
    ax.set_ylabel('Absolute Relative Difference (%)')
    ax.grid(b=True, which='both', color='grey', linestyle='-')
    ax.set_title('Duplicate type %s, n = %d' % (d, df[df.Type == d]['HOLEID'].count()))
    index += 1
    

Boxplots

In [11]:
fig, axes = plt.subplots(1,4, facecolor = 'w', sharey = True, figsize = (11,6))
# fig.set_figwidth(10)
boxprops = dict(color='grey')
meanpointprops = dict(marker='*', markeredgecolor='black',
                      markerfacecolor='firebrick', markersize = 8)
index = 0
for d in ['Field Duplicate', 'Coarse Reject Duplicate', 'Lab Duplicate', 'Umpire Lab']:
    ax = axes[index]
    data = np.array(df[df.Type == d][['Au_orig','Au_val']])
    ax.boxplot(data, labels = ['Original','Duplicate'],\
               showmeans=True, meanprops = meanpointprops,\
               boxprops = boxprops)
    ax.set_title(d)
    ax.set_yscale('log')
    if index == 0:
        ax.set_ylabel('Au gpt')
    yticks = ax.get_yticks()
    ax.set_yticklabels([y for y in yticks])        
    index += 1

Q-Q Plots

In [12]:
fig= plt.figure(facecolor = 'w', figsize = (12,12))
fig.suptitle('Q-Q plots for week of %s' % \
             (datetime.date.today() + datetime.timedelta(4 - \
              datetime.date.weekday(datetime.date.today()))).strftime('Friday, %B %d, %Y'), fontsize = 14)

index = 1

#create a range of numbers to get the percentiles for
linspace = 10 ** (np.linspace(-3, 0, 250))

for d in ['Field Duplicate', 'Coarse Reject Duplicate', 'Lab Duplicate', 'Umpire Lab']:
    ax = fig.add_subplot(2,2,index)
    
    # plot 1:1 line
    ax.plot((0,max(df[df.Type == d].Au_val)*1.1), (0,max(df[df.Type == d].Au_val)*1.1), 'r--')
    
    #plot data
    ax.plot(df[df.Type == d].quantile(linspace)['Au_val'], df[df.Type == d].quantile(linspace)['Au_orig'], 'k-')
    ax.plot(df[df.Type == d].quantile(linspace)['Au_val'], df[df.Type == d].quantile(linspace)['Au_orig'], 'r*')
    
    #set plot limits and scale
    limit = max(max(df[df.Type == d].Au_val), max(df[df.Type == d].Au_orig))
    xlim((0.005,limit))
    ylim((0.005, limit))
    ax.set_xscale('log')
    ax.set_yscale('log')
    xticks = ax.get_xticks()
    ax.set_xticklabels([x for x in xticks])
    yticks = ax.get_yticks()
    ax.set_yticklabels([y for y in yticks])
    
    #set axis and chart titles
    ax.set_xlabel('Duplicate (gpt)')
    ax.set_ylabel('Original (gpt)')
    ax.set_title('%s, n = %d' % (d, df[df.Type == d]['HOLEID'].count()))
    
    index += 1


In [13]:
fig= plt.figure(facecolor = 'w', figsize = (12,12))
fig.suptitle('Q-Q Plots for week of %s' % datetime.date.today().strftime('%B %d, %Y'), fontsize = 14)



#create a range of numbers to get the percentiles for
linspace = 10 ** (np.linspace(-3, 0, 100))
d = df[(df.Type == 'Coarse Reject Duplicate') | (df.Type == 'Field Duplicate')]

ax = fig.add_subplot(1,1,1)

# plot 1:1 line
ax.plot((0,max(d.Au_val)*1.1), (0,max(d.Au_val)*1.1), 'r--')

#plot data
ax.plot(d.quantile(linspace)['Au_val'], d.quantile(linspace)['Au_orig'], 'k-')
ax.plot(d.quantile(linspace)['Au_val'], d.quantile(linspace)['Au_orig'], 'r*')
#set plot limits and scale
limit = max(max(d.Au_val), max(d.Au_orig))
xlim((0.005,limit))
ylim((0.005, limit))
ax.set_xscale('log')
ax.set_yscale('log')

#set axis and chart titles
ax.set_xlabel('Duplicate')
ax.set_ylabel('Original')


    

In [14]:
axes

array([<matplotlib.axes._subplots.AxesSubplot object at 0x0C377350>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x0C63E6D0>], dtype=object)

In [36]:
df[df.Au_val > 4]

HOLEID  SAMPLEID      Ref        Type Au_gpt Au_gpt_orig       Date  \
95    UB17_420  S141217U  S141217  Umpire Lab   6.09        6.47 2017-07-18   
149  AKC17_123  V364077U  V364077  Umpire Lab  4.657        5.02 2017-06-13   
188   UB17_429  S141877U  S141877  Umpire Lab   5.22       0.034 2017-08-11   

     Au_val  Au_orig  Au_mean  Au_diff  
95    6.090    6.470   6.2800   -0.380  
149   4.657    5.020   4.8385   -0.363  
188   5.220    0.034   2.6270    5.186

In [51]:
df[(df.Au_val < 0.04) & (df.Au_orig >0.07) & (df.Type == 'Umpire Lab')]

HOLEID  SAMPLEID      Ref        Type Au_gpt Au_gpt_orig       Date  \
5    AKC17_123  V364217U  V364217  Umpire Lab  0.013       0.163 2017-06-13   
12   AKC17_127  V364557U  V364557  Umpire Lab  0.014       0.071 2017-07-18   
294  AKC17_123  V364057U  V364057  Umpire Lab  0.024       0.123 2017-06-13   

     Au_val  Au_orig  Au_mean  Au_diff  
5     0.013    0.163   0.0880   -0.150  
12    0.014    0.071   0.0425   -0.057  
294   0.024    0.123   0.0735   -0.099

In [12]:
fig= plt.figure(facecolor = 'w', figsize = (8,8))
fig.suptitle('Duplicate Analyses for week of %s' % \
             (datetime.date.today() + datetime.timedelta(4 - \
              datetime.date.weekday(datetime.date.today()))).strftime('Friday, %B %d, %Y'), fontsize = 14)


index = 1

for d in ['Umpire Lab']:
    ax = fig.add_subplot(1,1,1)
    ax.plot((0,max(df.Au_val)*2), (0,max(df.Au_val)*2), 'r-')
    
    m, b = np.polyfit(df[df.Type == d].Au_val, df[df.Type == d].Au_orig, 1)
    n = np.linspace(0.005, max(df.Au_val) * 1.1, 500)
    ax.plot(n, (n * m) + b, 'b--')
    ax.scatter(df[df.Type == d].Au_val, df[df.Type == d].Au_orig, c = 'k')
    limit = max(max(df.Au_val), max(df.Au_orig)) * 1.1
    ax.set_xlim((0.005,limit))
    ax.set_ylim((0.005, limit))
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Duplicate (gpt)', fontsize = 10)
    ax.set_ylabel('Original (gpt)', fontsize = 10)
    xticks = ax.get_xticks()
    ax.set_xticklabels([x for x in xticks])
    yticks = ax.get_yticks()
    ax.set_yticklabels([y for y in yticks])
    
    # calculate and display correlation
    r2 = df[df.Type == d].Au_val.corr(df[df.Type == d].Au_orig) **2
    xpos = 10 ** (np.log10(0.005) - ((np.log10(0.005) - np.log10(limit))* 0.1) )
    ypos = 10 ** (np.log10(0.005) - ((np.log10(0.005) - np.log10(limit))* 0.85) )
    ax.annotate(u'r² = %.3f' % (r2), (xpos, ypos) , bbox=dict(facecolor='w', alpha=0.2, ))
    xpos = 10 ** (np.log10(0.005) - ((np.log10(0.005) - np.log10(limit))* 0.5) )
    ypos = 10 ** (np.log10(0.005) - ((np.log10(0.005) - np.log10(limit))* 0.15) )
    ax.annotate(u'y = %.5f * x +\n %.5f' % (m, b), (xpos, ypos), bbox = dict(facecolor = 'b', alpha = 0.2, ), color = 'b')
    
    ax.set_title('Duplicate type %s, n = %d' % (d, df[df.Type == d]['HOLEID'].count()), fontsize = 12)
    index += 1
   

In [22]:
m = 1
b = -0.5
'y = {:.5f} * x {}\n {:.5f}'.format(m, '-' if b < 0 else '+',abs(b))

'y = 1.00000 * x -\n 0.50000'